In [1]:
from neo4j import GraphDatabase

In [2]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "12345678"
auth=(username, password)
driver = GraphDatabase.driver(uri, auth=(username, password))

In [3]:
node_id = "Disease::DOID:184"

In [4]:
def query1(driver, node_id):
    query = """
        MATCH (n {id: $node_id})
        OPTIONAL MATCH (n)-[r1]->(target)
        WHERE r1.metaedge IN ['DlA', 'DaG']
        OPTIONAL MATCH (source)-[r2]->(n)
        WHERE r2.metaedge IN ['CpD', 'CtD']
        RETURN n, COLLECT(DISTINCT target) AS target_nodes, COLLECT(DISTINCT source) AS source_nodes
    """
    with driver.session() as session:
        result = session.run(query, node_id=node_id)
        return result.data()  

In [ ]:
result = query1(driver, node_id)
result

[{'n': {'kind': 'Disease', 'name': 'bone cancer', 'id': 'Disease::DOID:184'},
  'target_nodes': [{'kind': 'Anatomy',
    'name': 'nose',
    'id': 'Anatomy::UBERON:0000004'},
   {'kind': 'Anatomy', 'name': 'appendage', 'id': 'Anatomy::UBERON:0000026'},
   {'kind': 'Anatomy', 'name': 'tendon', 'id': 'Anatomy::UBERON:0000043'},
   {'kind': 'Anatomy',
    'name': 'cranial nerve II',
    'id': 'Anatomy::UBERON:0000941'},
   {'kind': 'Anatomy', 'name': 'sternum', 'id': 'Anatomy::UBERON:0000975'},
   {'kind': 'Anatomy', 'name': 'leg', 'id': 'Anatomy::UBERON:0000978'},
   {'kind': 'Anatomy',
    'name': 'vertebral arch joint',
    'id': 'Anatomy::UBERON:0001067'},
   {'kind': 'Anatomy',
    'name': 'external carotid artery',
    'id': 'Anatomy::UBERON:0001070'},
   {'kind': 'Anatomy',
    'name': 'vertebral bone 2',
    'id': 'Anatomy::UBERON:0001093'},
   {'kind': 'Anatomy',
    'name': 'clavicle bone',
    'id': 'Anatomy::UBERON:0001105'},
   {'kind': 'Anatomy',
    'name': 'vertebral colum

In [13]:
names = []

disease_name = result[0]['n']['name']
names.append('Disease Name')
names.append(disease_name)

names.append('Anatomy and Genes')
for node in result[0]['target_nodes']:
    names.append(node['name'])

names.append('Compounds')
for node in result[0]['source_nodes']:
    names.append(node['name'])

names

['Disease Name',
 'bone cancer',
 'Anatomy and Genes',
 'nose',
 'appendage',
 'tendon',
 'cranial nerve II',
 'sternum',
 'leg',
 'vertebral arch joint',
 'external carotid artery',
 'vertebral bone 2',
 'clavicle bone',
 'vertebral column',
 'external acoustic meatus',
 'sacro-iliac joint',
 'epiphysis',
 'face',
 'arm',
 'elbow',
 'manual digit 1',
 'hip',
 'knee',
 'shoulder',
 'sternoclavicular joint',
 'glenohumeral joint',
 'sesamoid bone',
 'knee joint',
 'hip joint',
 'ankle joint',
 'elbow joint',
 'vertebral artery',
 'popliteal vein',
 'cheek',
 'olfactory nerve',
 'masseter muscle',
 'temporalis muscle',
 'maxillary artery',
 'ophthalmic artery',
 'trigeminal nerve',
 'abducens nerve',
 'facial nerve',
 'hypoglossal nerve',
 'occipital bone',
 'sphenoid bone',
 'temporal bone',
 'ethmoid bone',
 'nasal bone',
 'mandible',
 'hyoid bone',
 'external ear',
 'petrous part of temporal bone',
 'orbit of skull',
 'nasal cavity',
 'jaw skeleton',
 'lower jaw region',
 'eyelid',
 '

In [3]:
def query2(driver):
    query = """
    MATCH (disease)
    WHERE disease.kind = 'Disease'
    AND NOT EXISTS {
        MATCH (disease)-[]-(compound)
        WHERE compound.kind = 'Compound'
    }
    WITH COLLECT(disease) AS diseases
    MATCH (disease)-[disAnatRel]->(anatomy)
    WHERE disease IN diseases
      AND disAnatRel.metaedge = 'DlA' 
      AND anatomy.kind = 'Anatomy'
    WITH COLLECT(DISTINCT anatomy) AS anatomies
    MATCH (compound)-[compoundGeneRel]->(gene)<-[anatomyGeneRel]-(anatomy)
    WHERE compound.kind = 'Compound'
      AND gene.kind = 'Gene'
      AND anatomy IN anatomies
      AND (
          (compoundGeneRel.metaedge = 'CuG' AND anatomyGeneRel.metaedge = 'AdG')
          OR
          (compoundGeneRel.metaedge = 'CdG' AND anatomyGeneRel.metaedge = 'AuG')
      )
    
    RETURN DISTINCT compound
    """
    
    with driver.session() as session:
        result = session.run(query)
        return list(result.data())

In [4]:
compounds = query2(driver)

In [19]:
len(compounds)

783

In [20]:
compound_names = [item['compound']['name'] for item in compounds]
compound_names

['Quinacrine',
 'Topotecan',
 'Mitoxantrone',
 'Vorinostat',
 'Lovastatin',
 'Mitomycin',
 'Vindesine',
 'Vinorelbine',
 'Treprostinil',
 'Amlodipine',
 'Alprazolam',
 'Teniposide',
 'Chlorpromazine',
 'Raloxifene',
 'Triflupromazine',
 'Albendazole',
 'Toremifene',
 'Vincristine',
 'Vinblastine',
 'Nizatidine',
 'Astemizole',
 'Simvastatin',
 'Mebendazole',
 'Tamoxifen',
 'Trimipramine',
 'Irinotecan',
 'Perphenazine',
 'Clomifene',
 'Maprotiline',
 'Cerulenin',
 'Perhexiline',
 'Fluvastatin',
 'Pimozide',
 'Sertraline',
 'Amiodarone',
 'Podofilox',
 'Ciclopirox',
 'Clarithromycin',
 'Paclitaxel',
 'Chlorprothixene',
 'Alimemazine',
 'Docetaxel',
 'Colchicine',
 'Everolimus',
 'Thiothixene',
 'Niclosamide',
 'Crizotinib',
 'Dabrafenib',
 'Afatinib',
 'Flubendazole',
 'Azithromycin',
 'Cefotiam',
 'Cefalotin',
 'Entacapone',
 'Ciprofloxacin',
 'Metaxalone',
 'Minaprine',
 'Fulvestrant',
 'Nefazodone',
 'Estriol',
 'Temsirolimus',
 'Cyclosporine',
 'Pyrimethamine',
 'Reserpine',
 'Nitaz